In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/anuvaad-03/anuvaad_0.3/translator/fingerprint.pb
/kaggle/input/anuvaad-03/anuvaad_0.3/translator/saved_model.pb
/kaggle/input/anuvaad-03/anuvaad_0.3/translator/variables/variables.index
/kaggle/input/anuvaad-03/anuvaad_0.3/translator/variables/variables.data-00000-of-00001
/kaggle/input/file-name-20-02/file_name_20_0.2/translator/fingerprint.pb
/kaggle/input/file-name-20-02/file_name_20_0.2/translator/saved_model.pb
/kaggle/input/file-name-20-02/file_name_20_0.2/translator/variables/variables.index
/kaggle/input/file-name-20-02/file_name_20_0.2/translator/variables/variables.data-00000-of-00001
/kaggle/input/final-20-01-15e/translator1/fingerprint.pb
/kaggle/input/final-20-01-15e/translator1/saved_model.pb
/kaggle/input/final-20-01-15e/translator1/variables/variables.index
/kaggle/input/final-20-01-15e/translator1/variables/variables.data-00000-of-00001
/kaggle/input/transl-last-hour/translator1/fingerprint.pb
/kaggle/input/transl-last-hour/translator1/saved_model.pb
/kag

In [2]:
from nltk.translate.bleu_score import sentence_bleu

In [3]:
!pip install tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.8 MB/s eta 0:00:0000:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.79.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
apache-beam 2.44.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.


In [4]:
import tensorflow as tf
import tensorflow_text as tf_text

In [5]:
!pip install Metaphone
from metaphone import doublemetaphone

  Preparing metadata (setup.py) ... done
  Created wheel for Metaphone: filename=Metaphone-0.6-py3-none-any.whl size=13919 sha256=fd66140a2aa929bbcd9fed0a960ee73772d21e8b38f10b688284b4bd5286188c
  Stored in directory: /root/.cache/pip/wheels/3a/2c/57/cb7d38aa6c80a870a0341ccf4265c2921fb55b1f7f4b76b46b
Successfully built Metaphone


In [6]:
import pickle
with open('/kaggle/input/hin-vocab/vocab_data.pkl', 'rb') as fp:
    hin_vocab = pickle.load(fp)
vocab_keys=[l for l in hin_vocab]
len(hin_vocab),len(vocab_keys)

(26214, 26214)

In [7]:
import re
re.sub("[.!?\\-\'\"]", "", "    kaise ho tum viase .?,KSDFaha   ").lower().strip()

'kaise ho tum viase ,ksdfaha'

In [8]:
def t_text(line):
    line=re.sub("[.!?\\-\'\"]", "",line).lower().strip()
    string=''
    for j in line.split(' '):
        if doublemetaphone(j)[0]+'*'+doublemetaphone(j[::-1])[0]+'*'+j[:2]+'*'+j[len(j)-1:] in vocab_keys:
            string=string+list(hin_vocab[doublemetaphone(j)[0]+'*'+doublemetaphone(j[::-1])[0]+'*'+j[:2]+'*'+j[len(j)-1:]])[0]+' '
        else:
            string=string+j+' '
    return string.lower().strip()

In [9]:
import pandas as pd
data_val=pd.read_csv('/kaggle/input/validate-bleu/validation.tsv',sep='\t')
data_val=data_val[['cs_query','en_query']]
data_val=data_val.rename(columns={'cs_query':'Sentence','en_query':'trr'})
data_val['trr']=data_val['trr'].apply(lambda x: x.lower().strip())
data_val

,Sentence,trr
0,mere timer ko roko,pause my timer .
1,Please Liz ke liye video message banaye,please make video message for liz
2,Alarm create karo at 4 pm and 9 pm on tuesday,create an alarm at 4 pm and 9 pm on tuesday
3,Library events Cedar Park me hai,library events in cedar park
4,Please timer ko rokey,please pause timer
...,...,...
1385,Is summer ko outdoor concerts,outdoor concerts this summer
1386,Mai chahta hu ki aap mere alarm ko 10 minutes ...,i ' d like you to snooze my alarm for 10 minut...
1387,please Tupac play kare,please play tupac
1388,Rhonda ko message kare aur July me Vacation Bi...,message rhonda ask her details about vacation ...


In [10]:
'''three_input_text = tf.constant([t_text('sabse best friend'), 
                                t_text('That toh I know'),
                                t_text('She was bhunnoing the masalas jub phone ki ghuntee bugee'),
                                t_text('tum batao'),
                                t_text('kya karna hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhara papa ka naam kya hai'),
])

result = reloaded.translate(input_text=three_input_text)

for tr in result['text']:
  print(tr.numpy().decode())'''

"three_input_text = tf.constant([t_text('sabse best friend'), \n                                t_text('That toh I know'),\n                                t_text('She was bhunnoing the masalas jub phone ki ghuntee bugee'),\n                                t_text('tum batao'),\n                                t_text('kya karna hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhara papa ka naam kya hai'),\n])\n\nresult = reloaded.translate(input_text=three_input_text)\n\nfor tr in result['text']:\n  print(tr.numpy().decode())"

In [11]:
import numpy

In [12]:
def cal_bleu(x,y):
    return sentence_bleu([x.split()],y.split())

In [13]:
reloaded = tf.saved_model.load('/kaggle/input/transl-last-hour/translator1')
data_val['model_output1']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score1']=data_val[['trr','model_output1']].apply(lambda x: cal_bleu(*x),axis=1)
#data_val['bleu_score_inv1']=data_val[['trr','model_output1']].apply(lambda x: cal_bleu_inv(*x),axis=1)

/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [14]:
data_val

,Sentence,trr,model_output1,bleu_score1
0,mere timer ko roko,pause my timer .,my timer muffins .,0.707107
1,Please Liz ke liye video message banaye,please make video message for liz,please make an ozone liar for liz,0.660633
2,Alarm create karo at 4 pm and 9 pm on tuesday,create an alarm at 4 pm and 9 pm on tuesday,change the reminder for the nba flower . betwe...,0.272259
3,Library events Cedar Park me hai,library events in cedar park,library events cedar breath will only bring snow,0.481098
4,Please timer ko rokey,please pause timer,please pause timer remaining,0.707107
...,...,...,...,...
1385,Is summer ko outdoor concerts,outdoor concerts this summer,the concerts senior worse . . . .,0.594604
1386,Mai chahta hu ki aap mere alarm ko 10 minutes ...,i ' d like you to snooze my alarm for 10 minut...,,0.000000
1387,please Tupac play kare,please play tupac,please play tupac on your head,0.472871
1388,Rhonda ko message kare aur July me Vacation Bi...,message rhonda ask her details about vacation ...,the whole country while often ? ? and july oth...,0.246014


In [15]:
sentence_bleu(["i have eaten".split()], "i have eaten the food".split() )

/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.5623413251903491

In [16]:
data_val['bleu_score1'].mean()

0.48868569848849225

In [17]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-015-mixed-00409/translator1')
data_val['model_output2']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score2']=data_val[['trr','model_output2']].apply(lambda x: cal_bleu(*x),axis=1)


/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [18]:
data_val['bleu_score2'].mean()

0.5015653381706123

In [19]:
a=['please pause timer','please pause timer']
print(a[0]==a[1])
sentence_bleu(list(a[0].split()),a[1].split())

True


0

In [20]:
a=data_val[['trr','model_output1']].values[4]
print(a)
print(a[0]==a[1])
print(a[0])
print(a[1])
sentence_bleu(a[0].split(),a[1].split())

['please pause timer' 'please pause timer remaining']
False
please pause timer
please pause timer remaining


0

In [21]:
from statistics import mean

In [22]:
l1=[]
for i in data_val[['trr','model_output1']].values:
    l1.append(sentence_bleu(i[0].split(),i[1].split()))
mean(l1)

0.2028250632440598

In [23]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-02-15e/translator1')
data_val['model_output3']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score3']=data_val[['trr','model_output3']].apply(lambda x: cal_bleu(*x),axis=1)

In [24]:
data_val['bleu_score3'].mean()

0.4926671526466196

In [25]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-01-15e/translator1')
data_val['model_output4']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score4']=data_val[['trr','model_output4']].apply(lambda x: cal_bleu(*x),axis=1)


In [26]:
data_val['bleu_score4'].mean()

0.45489966622027306

In [27]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-15e/translator')
data_val['model_output5']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score5']=data_val[['trr','model_output5']].apply(lambda x: cal_bleu(*x),axis=1)


In [28]:
data_val['bleu_score5'].mean()

0.4957670827405976

In [29]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-22e/translator')
data_val['model_output6']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score6']=data_val[['trr','model_output6']].apply(lambda x: cal_bleu(*x),axis=1)


In [30]:
data_val['bleu_score6'].mean()

0.46744384825443563

In [31]:
reloaded = tf.saved_model.load('/kaggle/input/translator1/translator1')
data_val['model_output7']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score7']=data_val[['trr','model_output7']].apply(lambda x: cal_bleu(*x),axis=1)


In [32]:
data_val['bleu_score7'].mean()

0.4930093378903948

In [33]:
reloaded = tf.saved_model.load('/kaggle/input/translator-2/translator1')
data_val['model_output8']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score8']=data_val[['trr','model_output8']].apply(lambda x: cal_bleu(*x),axis=1)


In [34]:
data_val['bleu_score8'].mean()

0.48895535828532066

In [35]:
reloaded = tf.saved_model.load('/kaggle/input/change-vocab-20-01/change_vocab_20_0.1/translator')
reloaded.tf_translate(
                tf.constant([
                t_text('tumhara naam kya hai')
                        ]))['text'][0].numpy().decode()

'. . . . . kapil have done . . . .'

In [36]:
reloaded = tf.saved_model.load('/kaggle/input/change-vocab-20-01/change_vocab_20_0.1/translator')
data_val['model_output9']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score9']=data_val[['trr','model_output9']].apply(lambda x: cal_bleu(*x),axis=1)


In [37]:
data_val['bleu_score9'].mean()

0.42747776590630643

In [38]:
reloaded = tf.saved_model.load('/kaggle/input/change-vocab-20-01/change_vocab_20_0.1/translator')

three_input_text = tf.constant([t_text('tumhare papa kya kaam karte hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('mera naam rohan hai'),
                                t_text('tum batao'),
                                t_text('kya karna hai'),
])

result = reloaded.tf_translate(three_input_text)
print(result['text'])
for tr in result['text']:
  print(tr.numpy().decode())

tf.Tensor(
[b'. . . . . kapil you have gone . . .' b'you abb ? ?'
 b'what a tongue . . .'], shape=(3,), dtype=string)
. . . . . kapil you have gone . . .
you abb ? ?
what a tongue . . .


In [39]:
reloaded = tf.saved_model.load('/kaggle/input/file-name-20-02/file_name_20_0.2/translator')
data_val['model_output10']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score10']=data_val[['trr','model_output10']].apply(lambda x: cal_bleu(*x),axis=1)


In [40]:
data_val['bleu_score10'].mean()

0.36673934830470095

In [41]:
reloaded = tf.saved_model.load('/kaggle/input/anuvaad-03/anuvaad_0.3/translator')
data_val['model_output11']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score11']=data_val[['trr','model_output11']].apply(lambda x: cal_bleu(*x),axis=1)


In [42]:
data_val['bleu_score11'].mean()

0.23828332437177666

In [43]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-cv-20-15-14e/translator1')
data_val['model_output12']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score12']=data_val[['trr','model_output12']].apply(lambda x: cal_bleu(*x),axis=1)


In [44]:
data_val['bleu_score12'].mean()

0.4931502832943862

In [45]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-vocab-40k-mixed-batch/translator1')
data_val['model_output13']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score13']=data_val[['trr','model_output13']].apply(lambda x: cal_bleu(*x),axis=1)


In [46]:
data_val['bleu_score13'].mean()

0.49081807985666515

In [47]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-vocab-40k-mixed-batch-v2/translator1')
data_val['model_output14']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score14']=data_val[['trr','model_output14']].apply(lambda x: cal_bleu(*x),axis=1)


In [48]:
data_val['bleu_score14'].mean()

0.49951374499784634

In [49]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-vocab-53k-mixed-batch/translator1')
data_val['model_output15']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score15']=data_val[['trr','model_output15']].apply(lambda x: cal_bleu(*x),axis=1)


In [50]:
data_val['bleu_score15'].mean()

0.5253561307063519

In [51]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-vocab-53k-mixed-batch-v2/translator1')
data_val['model_output16']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score16']=data_val[['trr','model_output16']].apply(lambda x: cal_bleu(*x),axis=1)


In [52]:
data_val['bleu_score16'].mean()

0.5253726185106636

In [53]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-vocab-53k-16bs-mixed-batch/translator1')
data_val['model_output17']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score17']=data_val[['trr','model_output17']].apply(lambda x: cal_bleu(*x),axis=1)


In [54]:
data_val['bleu_score17'].mean()

0.5024274764866679

In [55]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-vocab-53k-16bs-mixed-batch-v2/translator1')
data_val['model_output18']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score18']=data_val[['trr','model_output18']].apply(lambda x: cal_bleu(*x),axis=1)


In [56]:
data_val['bleu_score18'].mean()

0.4925301116738335

In [57]:
reloaded = tf.saved_model.load('/kaggle/input/human-anno-mixed-batch-v2/translator1')
data_val['model_output19']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score19']=data_val[['trr','model_output19']].apply(lambda x: cal_bleu(*x),axis=1)


In [58]:
data_val['bleu_score19'].mean()

0.5486404822217614

In [59]:
reloaded = tf.saved_model.load('/kaggle/input/human-anno-mixed-batch/translator1')
data_val['model_output20']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score20']=data_val[['trr','model_output20']].apply(lambda x: cal_bleu(*x),axis=1)


In [60]:
data_val['bleu_score20'].mean()

0.5541337440578491

In [61]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-200k-mixed-batch-v2/translator1')
data_val['model_output21']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score21']=data_val[['trr','model_output21']].apply(lambda x: cal_bleu(*x),axis=1)


In [62]:
data_val['bleu_score21'].mean()

0.5145423097108061

In [63]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-200k-mixed-batch/translator1')
data_val['model_output22']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score22']=data_val[['trr','model_output22']].apply(lambda x: cal_bleu(*x),axis=1)


In [64]:
data_val['bleu_score22'].mean()

0.523369249672173

In [65]:
data_val

,Sentence,trr,model_output1,bleu_score1,model_output2,bleu_score2,model_output3,bleu_score3,model_output4,bleu_score4,...,model_output18,bleu_score18,model_output19,bleu_score19,model_output20,bleu_score20,model_output21,bleu_score21,model_output22,bleu_score22
0,mere timer ko roko,pause my timer .,my timer muffins .,0.707107,mute my timer,0.544446,"my timer , at seconds",0.562341,pause my timer .,1.000000,...,my timer figure credit !,0.562341,pause my timer .,1.000000,sun makes my eyes !,0.668740,pause timer .,0.602529,pause timer .,0.602529
1,Please Liz ke liye video message banaye,please make video message for liz,please make an ozone liar for liz,0.660633,liz s pitch cookie time please .,0.731110,"please video message now , please .",0.516973,please make burnol s keep coming from the sand...,0.323772,...,please rai of liz s want to be killed,0.686589,please run by apna,0.428882,please run by depositing attention,0.547518,"please man , washington gas ! ! !",0.594604,please liz man started travelling to receive .,0.707107
2,Alarm create karo at 4 pm and 9 pm on tuesday,create an alarm at 4 pm and 9 pm on tuesday,change the reminder for the nba flower . betwe...,0.272259,create got an alarm set for pm and pm on tuesd...,0.394588,alarm needs to exercise at pm and pm on tuesday .,0.381633,flag apne apne ka kuch time . . . . pm and pm ...,0.169273,...,create a reminder karte about the kayak at pm ...,0.332602,create an alarm at pm and pm on tuesday,0.385233,create an alarm at pm and pm on tuesday,0.385233,sit ramesh wake up at pm and pm on tuesday,0.359795,stuck at pm on tuesday and pm on tuesday .,0.325112
3,Library events Cedar Park me hai,library events in cedar park,library events cedar breath will only bring snow,0.481098,"library is at the library events . , , ,",0.386097,change the library events in the library showi...,0.379918,put the temperate in the library events on tha...,0.345721,...,library events go in delhi . . . at least,0.427287,library events in cedar park,1.000000,library events in cedar park,1.000000,give me the traffic in cedar events,0.516973,live events in cedar neighborhood,0.562341
4,Please timer ko rokey,please pause timer,please pause timer remaining,0.707107,please pause the timer,0.707107,please pause timer once,0.707107,please stop a timer now . . .,0.707107,...,please pause the timer .,0.622333,please pause timer .,0.707107,please pause timer,1.000000,please pause timer,1.000000,please pause the timer .,0.622333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1385,Is summer ko outdoor concerts,outdoor concerts this summer,the concerts senior worse . . . .,0.594604,outdoor concerts this summer,1.000000,story summer . what type of parties are i,0.577350,hank likes concerts this summer,0.562341,...,are there summer lighting is this summer,0.467138,outdoor concerts this summer,1.000000,outdoor concerts this summer,1.000000,outdoor concerts this summer,1.000000,outdoor concerts this summer,1.000000
1386,Mai chahta hu ki aap mere alarm ko 10 minutes ...,i ' d like you to snooze my alarm for 10 minut...,,0.000000,i may have done faith in my name . . . . . giv...,0.600964,"i like you , snooze my alarm for the poet snoo...",0.203334,i would like you to be reminded of my yearly p...,0.231245,...,"while leaving the logic is , can you snooze my...",0.400160,get an alarm set for the forty days,0.378487,alarm for mera birthday . m kiya ! was think p...,0.338820,i am raju laughing till i will tell you to plz...,0.400160,i am raju laughing to allah make you by callin...,0.707107
1387,please Tupac play kare,please play tupac,please play tupac on your head,0.472871,please play tupac please play me,0.472871,please play tupac s . the data .,0.365555,please play tupac on the other team .,0.365555,...,please play tupac,1.000000,please play tupac,1.000000,please play tupac,1.000000,please play tupac,1.000000,please play tupac,1.000000
1388,Rhonda ko message kare aur July me Vacat

In [66]:
'ok'

'ok'

In [67]:
data_val.to_csv('bleu_score.csv', index=False)

In [69]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-200k-mixed-batch/translator1')

three_input_text = tf.constant([t_text('tumhare papa kya kaam karte hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('mera naam rohan hai'),
                                t_text('tum batao'),
                                t_text('kya karna hai'),
])

result = reloaded.tf_translate(three_input_text)
print(result['text'])
for tr in result['text']:
  print(tr.numpy().decode())

tf.Tensor(
[b'what is the concern , i will work at least !'
 b'what is the name of the parking among the lake'
 b'my name is necessary to be' b'you have to tell you'
 b'what a member of the farmer'], shape=(5,), dtype=string)
what is the concern , i will work at least !
what is the name of the parking among the lake
my name is necessary to be
you have to tell you
what a member of the farmer


In [70]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-200k-mixed-batch-v2/translator1')
three_input_text = tf.constant([t_text('tumhare papa kya kaam karte hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('mera naam rohan hai'),
                                t_text('tum batao'),
                                t_text('kya karna hai'),
])

result = reloaded.tf_translate(three_input_text)
print(result['text'])
for tr in result['text']:
  print(tr.numpy().decode())

tf.Tensor(
[b'should there be less kumar chaturvedi . . . downtown ? ?'
 b'what is the name of the daughters mans ?' b'my name is done by p .'
 b'you say' b'is the sting to curry'], shape=(5,), dtype=string)
should there be less kumar chaturvedi . . . downtown ? ?
what is the name of the daughters mans ?
my name is done by p .
you say
is the sting to curry


In [71]:
reloaded = tf.saved_model.load('/kaggle/input/human-anno-mixed-batch/translator1')
three_input_text = tf.constant([t_text('tumhare papa kya kaam karte hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('mera naam rohan hai'),
                                t_text('tum batao'),
                                t_text('kya karna hai'),
])

result = reloaded.tf_translate(three_input_text)
print(result['text'])
for tr in result['text']:
  print(tr.numpy().decode())

tf.Tensor(
[b'what to drink sunday jo tak .'
 b'if the rascal has gone to airtel , will he make it to bring ?'
 b'my torture is still . .' b'you will be shaken'
 b'when is it going to start ?'], shape=(5,), dtype=string)
what to drink sunday jo tak .
if the rascal has gone to airtel , will he make it to bring ?
my torture is still . .
you will be shaken
when is it going to start ?


In [72]:
reloaded = tf.saved_model.load('/kaggle/input/human-anno-mixed-batch-v2/translator1')
three_input_text = tf.constant([t_text('tumhare papa kya kaam karte hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('mera naam rohan hai'),
                                t_text('tum batao'),
                                t_text('kya karna hai'),
])

result = reloaded.tf_translate(three_input_text)
print(result['text'])
for tr in result['text']:
  print(tr.numpy().decode())

tf.Tensor(
[b'when asked to me . . . . ?' b'if you want to ask shopping ? ? ?'
 b'my torture is still on .' b'you will get pakistanis'
 b'what to do with work'], shape=(5,), dtype=string)
when asked to me . . . . ?
if you want to ask shopping ? ? ?
my torture is still on .
you will get pakistanis
what to do with work


In [73]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-vocab-53k-mixed-batch-v2/translator1')
three_input_text = tf.constant([t_text('tumhare papa kya kaam karte hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('mera naam rohan hai'),
                                t_text('tum batao'),
                                t_text('kya karna hai'),
])

result = reloaded.tf_translate(three_input_text)
print(result['text'])
for tr in result['text']:
  print(tr.numpy().decode())

tf.Tensor(
[b'. . . i will setup a vein ! ! !'
 b'it is called brother , with mothers tomrrow .'
 b'name the sound of the wrists .' b'you got shaken'
 b', do you make me cry ?'], shape=(5,), dtype=string)
. . . i will setup a vein ! ! !
it is called brother , with mothers tomrrow .
name the sound of the wrists .
you got shaken
, do you make me cry ?


In [74]:
reloaded = tf.saved_model.load('/kaggle/input/all-data-vocab-53k-mixed-batch/translator1')
three_input_text = tf.constant([t_text('tumhare papa kya kaam karte hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('mera naam rohan hai'),
                                t_text('tum batao'),
                                t_text('kya karna hai'),
])

result = reloaded.tf_translate(three_input_text)
print(result['text'])
for tr in result['text']:
  print(tr.numpy().decode())

tf.Tensor(
[b'it means your head . . will you become cheap ?'
 b'it is called kapare pin ! !' b'name of sir ?' b'you will get up !'
 b'by doing half'], shape=(5,), dtype=string)
it means your head . . will you become cheap ?
it is called kapare pin ! !
name of sir ?
you will get up !
by doing half


In [75]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-015-mixed-00409/translator1')
three_input_text = tf.constant([t_text('tumhare papa kya kaam karte hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('mera naam rohan hai'),
                                t_text('tum batao'),
                                t_text('kya karna hai'),
])

result = reloaded.tf_translate(three_input_text)
print(result['text'])
for tr in result['text']:
  print(tr.numpy().decode())

tf.Tensor(
[b'your father is happy always' b'your father has a little of you'
 b'serious problem is the mind' b'you holy' b'serious problem'], shape=(5,), dtype=string)
your father is happy always
your father has a little of you
serious problem is the mind
you holy
serious problem


In [68]:
import time
for i in range(1000):
    time.sleep(4)

KeyboardInterrupt: 